In [1]:
from google.colab import files
files.upload() # kaggle.jsonをアップロード
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
import pandas as pd
import numpy as np
# 四分位範囲を求めるためのクラスをインポート
from scipy import stats
# データの準備
def prepare():
    !kaggle datasets download -d rajyellow46/wine-quality
    !unzip wine-quality.zip
# データの前処理（データフレームに読み込む）
def preprocess():
    df = pd.read_csv('winequalityN.csv')
    # 分析対象とする列
    fields = ['fixed acidity', 
              'volatile acidity', 
              'citric acid', 
              'residual sugar', 
              'chlorides', 
              'free sulfur dioxide', 
              'total sulfur dioxide', 
              'density', 
              'pH', 
              'sulphates', 
              'alcohol','quality']

    # 読み込んだデータの行数
    print('Num. of data: %d' % len(df))
    data = df.loc[:,fields]
    df = pd.DataFrame(data, columns=fields)
    return data, df, fields

# 外れ値検出（範囲指定）
def detect_outlier_range(df, fields):
    # 絶対値が7.0より大きい値を含む行を検出
    k = 7.0
    outlier = df[(np.abs(df)>k).any(1)]
    print('Outliers (%d):' % len(outlier))
    odf = pd.DataFrame(outlier, columns=fields)
    print(odf[:5]) 

# 四分位範囲(IQR)による外れ値除去
def detect_outlier_IQR(o_df, fields, target_col): 
    df = o_df.loc[:, [target_col]]
    # 第1四分位数（=25パーセンタイル）
    q1 = stats.scoreatpercentile(df, 25)
    # 第3四分位数（=75パーセンタイル）
    q3 = stats.scoreatpercentile(df, 75)
    # IQRの計算
    iqr = q3 - q1 
    # 外れ値の範囲を計算する
    # 第1四分位数(q1)からiqr*1.5を引く
    iqr_min = q1 - iqr * 1.5  
    # 第3四分位数(q3)にiqr*1.5を足す
    iqr_max = q3 + iqr * 1.5 
    print('Range(Min:{:.3f}, Max:{:.3f})'.format(
iqr_min, iqr_max))

    # 範囲から外れている値を除外したデータフレームを作成
    iqr_fsd_df = o_df[
(df[target_col]<iqr_max) \
 & (df[target_col]>iqr_min)]
    iqr_outliers = o_df[
  (df[target_col]>=iqr_max) \
  | (df[target_col]<=iqr_min)]
    print('Remove outliers by \
          IQR({})'.format(len(iqr_fsd_df)))
    print(iqr_fsd_df[20:31])
    print('Records including Outliers({}):'.format(
                                 len(iqr_outliers)))
    print(iqr_outliers.head())
    return iqr_fsd_df

def main():
    prepare()
    data, df, fields = preprocess()
    detect_outlier_range(df, fields)
    # 2つの列に着目し、順番に外れ値除去を行う
    iqr_fsd_df = detect_outlier_IQR(df, fields, 
                              'total sulfur dioxide')
    detect_outlier_IQR(iqr_fsd_df, fields, 'alcohol')

if __name__ == '__main__':
    main()
    

  0% 0.00/98.0k [00:00<?, ?B/s]
100% 98.0k/98.0k [00:00<00:00, 36.3MB/s]
Archive:  wine-quality.zip
  inflating: winequalityN.csv        
Num. of data: 6497
Outliers (6497):
   fixed acidity  volatile acidity  citric acid  ...  sulphates  alcohol  quality
0            7.0              0.27         0.36  ...       0.45      8.8        6
1            6.3              0.30         0.34  ...       0.49      9.5        6
2            8.1              0.28         0.40  ...       0.44     10.1        6
3            7.2              0.23         0.32  ...       0.40      9.9        6
4            7.2              0.23         0.32  ...       0.40      9.9        6

[5 rows x 12 columns]
Range(Min:-41.500, Max:274.500)
Remove outliers by           IQR(6487)
    fixed acidity  volatile acidity  citric acid  ...  sulphates  alcohol  quality
20            6.2              0.66         0.48  ...       0.39     12.8        8
21            6.4              0.31         0.38  ...       0.35     11.0 